In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
# import plotly.graph_objects as go
import plotly.express as px
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, f1_score, auc

import statsmodels.api as sm
import statsmodels.formula.api as smf

import folium
from folium.plugins import HeatMapWithTime
from folium.plugins import HeatMap
from datetime import datetime
from branca.colormap import linear
from matplotlib import cm

import geopy.distance
import requests
import json

In [ ]:
data_path = '../input/berlin-airbnb/'

# geojson from https://github.com/ljwolf/geopython/blob/master/data/berlin-neighbourhoods.geojson
geo_df = gpd.read_file(data_path + 'neighbourhoods.geojson')
listings_df = pd.read_csv(data_path + 'listings.csv', low_memory=False)
reviews_df = pd.read_csv(data_path + 'reviews.csv',parse_dates = [2])

# Question 1:
What proportion of Airbnb hosts in Berlin likely use hosting as a primary source of income (or are businesses)? 

## Data Preparation:

In [ ]:
# listings_df.groupby('calculated_host_listings_count')['room_type'].agg(set).values

In [ ]:
availability_room_type = pd.DataFrame(listings_df.groupby('room_type')['availability_365'].mean()).reset_index()
availability_room_type['availability_365'] = availability_room_type['availability_365'].apply(lambda x: round(x))
fig = px.bar(availability_room_type, x='room_type', y='availability_365',
      hover_data = ['availability_365'], labels = {'labels': 'Type of Listings',
                                                    'availability_365': 'Average Availability per Year'})
fig.update_layout(
    yaxis_title="Average Number of Days Available in Year",
    xaxis_title = "Type of Listings",
    title = "Average Availability of Listings Based on Type of Listing")

fig.show()

In [ ]:
df = pd.DataFrame([('Entire home/apt',85),
        ('Hotel room',275),
        ('Private room',55),
        ('Shared room',114)])
df =df.rename(columns={0:'room_type',1:'availability_365'})

In [ ]:
availability_room_type

In [ ]:
# clean the price column (change to numeric)
listings_df['price'] = pd.to_numeric(listings_df.price.apply(
    lambda x: x.replace("$", "").replace(",","")))
room_types = pd.merge(listings_df.room_type.value_counts(), listings_df.groupby('room_type').agg({'price': 'mean'})['price'], left_index=True, right_index=True).reset_index()
room_types['Price'] = room_types['price'].apply(lambda x: round(x))

fig = go.Figure(
    data=[
        go.Bar(name='No. of Listings', x=room_types['index'], y=room_types['room_type'], yaxis='y', offsetgroup=1),
        go.Bar(name='Price Paid', x=room_types['index'], y=room_types['Price'], yaxis='y2', offsetgroup=2)
    ],
    layout={
        'yaxis': {'title': 'Number of Listings'},
#         'xaxis' : {"Room Type"},
        'yaxis2': {'title': 'Price Paid( $ )', 'overlaying': 'y', 'side': 'right'}
    }
)

# Change the bar mode
fig.update_layout(
    xaxis_title = "Room Type"
    )
fig.update_layout(barmode='group')
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Number_of_Listings_Relative_to_Avg_Price_each_Room_Type.html')

In [ ]:
room_types

A host can have multiple entries in listing_df if his/her listings is also present in different neighbourhood_cleansed

In [ ]:
listings_df[listings_df.duplicated('host_id')][['host_id', 'calculated_host_listings_count', 'neighbourhood_cleansed']].sort_values('host_id')

In [ ]:
def make_listings_info_df(bins, labels, hosts = False):
    '''
    Function to make a dataframe with average values of availability and 
    listing numbers based on the bins chosen
    Inputs: bins - a pd.IntervalIndex (pandas IntervalIndex) consisting of the cuts you 
                    want to divide the listing
                    counts into
            labels - a List of strings with each corresponding to each provided 
                        interval in 'bins'
            hosts - If False (default), just get the percentage of listings that
                    correspond to the bin, e.g. if bin is 2 listings, then 
                    get information for all listings that have a host with 2 listings.
                    If True, get information for hosts, e.g. if bin is 2 listings,
                    get information for all hosts that have 2 listings.
    Output: a Pandas dataframe containing the labels and bins with the corresponding 
            percentage of listings (or hosts if hosts=True) in each bin along
            with the average availability (/365 days)
            of all the listings in that bin
    '''
    if hosts:
#         df = listings_df.groupby('host_id').agg({'calculated_host_listings_count':'sum', 'availability_365':'mean'})
        df = listings_df.groupby('host_id')[['calculated_host_listings_count',
                                             'availability_365']].mean()
#         print(df['calculated_host_listings_count'].value_counts())
        
    else:
        df = listings_df.copy()
        
#    print(df.shape)
        
    # make a new column with the group/cut that the listing belongs to (based on the number of 
    # listings the host has)
    
    # calculated_host_listings_count: The number of listings(total number of rooms a host has) the host has in the current scrape, in the city/region geography.
    df['num_listings_cut'] = pd.cut(df.calculated_host_listings_count, bins)

    # get percentage of listings that are the only listing vs. one of many listings
    num_listings_percent = df.num_listings_cut.value_counts(normalize=True)

    # get the average availability out of the calendar year that each category offers
    availability_num_listings = pd.DataFrame(df.groupby('num_listings_cut')['availability_365'].mean()).reset_index()

    # make dataframe that contains the listing percentages by type of host (host listing num) 
    # using 'num_listings_perc'
    df = pd.DataFrame(num_listings_percent).reset_index().rename(columns = {'index': 'cut', 
                                                                            'num_listings_cut': 'perc_listings'})
    # make sure the categories/cuts are in order
    df = df.sort_values(by='cut').reset_index(drop=True) 

    # merge the string version of groups/cuts into dataframe:
    df = pd.concat([labels, df], axis=1).rename(columns = {0: 'labels'})

    # include the average availability for each of these cuts into the dataframe 
    df = df.merge(availability_num_listings, left_on='cut', 
                           right_on='num_listings_cut').drop(columns = 'num_listings_cut')
    
    # make a new column for the listing percentages cleaned (rounded to 4 digits and in percentage format)
    df['perc_listings_clean'] = df.perc_listings.apply(
        lambda x: 100*round(x,4))
    
    return df

### Results/ Visualisations:

In [ ]:

# split the total number of listings that the host has into different groups
bins = pd.IntervalIndex.from_tuples([(-1, 1), (1, 3),(3,10),(10,100)])

# make a list with strings representing each of the cuts -- the interval is (closed, open] 
# meaning (2,3] contains only 3
labels = pd.Series(['1 Listing','2-3 Listings','4-10 Listings','11+ Listings'])

# create the dataframe with information about listings based on 
# how many listings the host has
listings_info_df = make_listings_info_df(bins, labels)
# rename some of the columns to make it more readable in the plot
listings_info_df = listings_info_df.rename(columns = {'labels': 'Host Listings',
                                                     'perc_listings': 'Percentage of Listings'})
listings_info_df

In [ ]:
# make a pie chart of the breakdown of listings
fig = px.pie(listings_info_df, names = 'Host Listings', values='Percentage of Listings', 
             title='Breakdown of Listings by Host\'s Total Number of Listings(Area Based)',
            color_discrete_sequence = px.colors.sequential.Emrld,
             hover_data=['Host Listings'], labels={'Host Listings':'Host Listings'})
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Breakdown_of_Listings.html')

# Now group the dataframe of listings based on the number of listings each host has, and look at the mean 
# number of listings in each category
hosts_df = listings_df.groupby('host_id')[['calculated_host_listings_count',
                                           'availability_365']].mean()
hosts_df['num_listings_cut'] = pd.cut(hosts_df.calculated_host_listings_count, bins)
hosts_df.groupby('num_listings_cut')['calculated_host_listings_count'].mean()

In [ ]:
df = pd.DataFrame([('1 Listing','(-1, 1]',0.736727,48.878850,73.67),
                    ('2-3 Listings','(1, 3]',0.162454,91.673924,16.25),
                    ('4-10 Listings','(3, 10]',0.063861,206.164281,6.39),
                    ('11+ Listings','(10, 100]',0.036958,236.570968,3.70)])
# df.columns = {"Host Listings",'cut','Percentage of Listings','availability_365','perc_listings_clean'}
# # rename some of the columns to make it more readable in the plot
df = df.rename(columns={0:"Host Listings",1:'cut',2:'Percentage of Listings',3:'availability_365',4:'perc_listings_clean'})
df

In [ ]:
df

In [ ]:
# Still grouping the dataframe on the basis of hosts (not listings) we create 
# a dataframe containing 
# information about the hosts based on how many listings they have

# define the intervals we want to split on
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 3), (3,100)])
# associate bins with labels
labels = pd.Series(['1 Listing','2-3 Listings','4+ Listings'])

# make the host info dataframe
hosts_info_df = make_listings_info_df(bins, labels, hosts=True)
print(hosts_info_df)
# plot the host info dataframe based on how many listings the different categories of hosts have

fig = px.pie(hosts_info_df, names = 'labels', values='perc_listings_clean', 
             title='Breakdown of Hosts by Number of Listings',
            color_discrete_sequence = px.colors.sequential.Emrld,
             #hover_name = 'labels',
             hover_data = ['perc_listings_clean'],
            labels ={'labels': 'Num Listings','perc_listings_clean': 'Percentage'})
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Breakdown_of_Listings.html')

In [ ]:
# Make the same hosts info dataframe but for different intervals to look at 
# average availability of listings
# throughout the year

bins = pd.IntervalIndex.from_tuples([(-1, 1), (1, 2), (2,3),(3,4),(4, 100)])

labels = pd.Series(['1 Listing','2 Listings','3 Listings','4 Listings','5+ Listings'])

# make the host info dataframe
hosts_info_df = make_listings_info_df(bins, labels, hosts=True)
# change formatted percentage of listings for input into the bar plot
hosts_info_df['perc_listings_clean'] = hosts_info_df.perc_listings.apply(
    lambda x: "Represents {:.2f}% of Listings".format(x*100))
hosts_info_df['availability_365'] = hosts_info_df['availability_365'].apply(lambda x: round(x))
print(hosts_info_df)
# plot results using Plotly express

fig = px.bar(hosts_info_df, x='labels', y='availability_365',
      hover_data = ['perc_listings_clean'], labels = {'labels': 'Num Listings',
                                                    'availability_365': 'Average Availability per Year',
                                                   'perc_listings_clean': 'Percentage'})
fig.update_layout(
    yaxis_title="Average Number of Days Available in Year",
    xaxis_title = "Host's Number of Listings",
    title = "Host's Average Availability by Number of Listings")

fig.show()
# Saving the plot
plotly.offline.plot(fig, filename='Hosts_Average_Availability.html')

In [ ]:
df = pd.DataFrame([('1 Listing','(-1, 1]',0.893015,49,'Represents 89.30% of Listings'),
        ('2 Listings','(1, 2]',0.075193,85,'Represents 7.52% of Listings'),
        ('3 Listings''(2, 3]',0.015511,114,'Represents 1.55% of Listings'),
        ('4 Listings','(3, 4]',0.005684,179,'Represents 0.57% of Listings'),
        ('5+ Listings','(4, 100]',0.010597,225,'Represents 1.06% of Listings')])
df = df.rename(columns={0: 'labels',1:'cut',2:'perc_listings',3:'availability_365',4:'perc_listings_clean'})

df

In [ ]:
hosts_info_df

Avaliability_x. The availability of the listing x days in the future as determined by the calendar. **NOTE** a listing may not be available because it has been booked by a guest or blocked by the host.

# Question 2:
How does the price of an Airbnb differ throughout neighbourhoods in Berlin, and what neighbourhoods have the best price for value?

## Data Understanding:

In [ ]:
listings_df.shape, reviews_df.shape

In [ ]:
# clean the price column (change to numeric)
# listings_df['price'] = pd.to_numeric(listings_df.price.apply(
#     lambda x: x.replace("$", "").replace(",","")))
# look at breakdown of prices in different districts
listings_df.groupby('neighbourhood_group_cleansed').agg({'price':['mean',
                                                                  'median','std','count']})

## Data Preparation:

In [ ]:
# listings_df[listings_df.name.str.contains('zentral gelegen|WG|Axel|Friedrichshain|Hackescher') == True].price.unique()

In [ ]:
fig = px.scatter(listings_df, y="price", title="Overall Price Distribution With Outliers")
# fig.title("Price distribution") 
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Overall_Price_Distribution_With_Outliers.html')

In [ ]:
# Get the year and month that each review was made in new columns, so we can split by date easier

reviews_df['year'] = reviews_df.date.dt.year
reviews_df['month'] = reviews_df.date.dt.month
reviews_df = reviews_df.sort_values(by='date', ascending=True).reset_index(drop=True)
# CLEAN the listings_df price data, done mostly by investigating manually the 
# extremely expensive outliers 
# some hotels and hostels list all listings as 1000 or 999 a night, so we
# removed these, as well with some individual shared rooms listing their monthly prices


# set these falsly labelled property prices to nan
listings_df.loc[(listings_df.price > 2000) & 
                (listings_df.name.str.contains('zentral gelegen|WG|Axel|Friedrichshain|Hackescher|Zentrum')),
                'price'] = np.nan

# set the dorm rooms prices to nan, as they do not cost 1000 a night
listings_df.loc[(listings_df.price == 1000) & (listings_df.name.str.contains('Dorm')),'price'] = np.nan

# target individual properties that was not accurately priced
listings_df.loc[(listings_df.price == 1000) & 
                (listings_df.name.str.contains('Generator|Private|MAMA|Quiet')), 'price'] = np.nan

listings_df.loc[(listings_df.price == 1000) & 
                (listings_df.summary.str.contains('Base yourself')), 'price'] = np.nan

# found all those priced at 999 dollars were not accurate
listings_df.loc[(listings_df.price == 999),'price'] = np.nan
# can see that all the remaining properties above 900$ a night are accurately
# priced, so we can take averages without including false outliers
listings_df[listings_df.price > 900].head()

In [ ]:
fig = px.scatter(listings_df, y="price", title="Overall Price Distribution After Removing Outliers")
# fig.title("Price distribution") 
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Overall_Price_Distribution_After_Removing_Outliers.html')

In [ ]:
# create dataframe combining the reviews and certain information from the listings dataframe,
# so now we have a dataframe only with listings that have been reviewed at least once

reviews_neighbourhood = reviews_df[['listing_id',
                                    'comments',
                                    'date','year',
                                    'month']].copy().merge(listings_df[['id',
                                                                        'neighbourhood_group_cleansed',
                                                                        'neighbourhood_cleansed',
                                                                        'price','name',
                                                                        'space','summary',
                                                                        'latitude','longitude']].copy(),
                                                                  left_on='listing_id',
                                                                  right_on='id',
                                                                  how='inner').drop(columns = 'id')
# take only those reviews from 2019 and later so we have accurate prices paid
# make a dataframe with all reviews and (approximate) prices paid for those stays and group by 
# neighbourhood

# FOR THE neighbourhood_cleansed
reviews_grouped = reviews_neighbourhood[reviews_neighbourhood.year >= 
                                        2019].groupby(['neighbourhood_cleansed'])[['price']].agg(['mean','median',
                                                                                                  'count'])
# round the price to 2 decimal points
reviews_grouped['price']  = reviews_grouped.price.apply(lambda x: round(x,2))
# clean up the column names
reviews_grouped.columns = ['_'.join(col).strip() for col in reviews_grouped.columns.values]

# add neighbourhood_group_cleansed

reviews_grouped = reviews_grouped.merge(reviews_neighbourhood[['neighbourhood_cleansed', 'neighbourhood_group_cleansed']].drop_duplicates(), 
                                        left_on = 'neighbourhood_cleansed', right_on='neighbourhood_cleansed')

# if a neighbourhood has too little reviews to take the mean price paid, set to NAN so it's not included in map
reviews_grouped.loc[reviews_grouped.price_count < 40,'price_mean'] = np.nan

reviews_grouped.reset_index(inplace=True)

print(reviews_grouped.head(), reviews_grouped.shape)

## FOR THE neighbourhood_grouped_cleansed

# reviews_grouped_1 = reviews_neighbourhood[reviews_neighbourhood.year 
#                       >=2019].groupby('neighbourhood_group_cleansed')['price'].agg('mean').reset_index()

# reviews_grouped_1 = reviews_grouped_1.merge(reviews_neighbourhood[['neighbourhood_group_cleansed', 'neighbourhood_cleansed']], 
#                                             left_on='neighbourhood_group_cleansed', right_on='neighbourhood_group_cleansed')

# reviews_grouped_1 = reviews_grouped_1.drop_duplicates('neighbourhood_cleansed').reset_index(drop=True)

# reviews_grouped_1['price_mean'] = reviews_grouped_1.price.apply(lambda x: round(x,2))

# print(reviews_grouped_1.head(), reviews_grouped_1.shape)

In [ ]:
# listings_df.neighbourhood_cleansed.unique(), listings_df.neighbourhood_group_cleansed.unique()

In [ ]:
# make the colormap used for the folium plot of neighbourhood prices
colormap = linear.OrRd_09.scale(
    reviews_grouped.price_mean.min(),
    reviews_grouped.price_mean.max())
# make a dataframe that combines the geopandas df with the reviews_grouped dataframe

gdf_price = geo_df.merge(reviews_grouped, left_on='neighbourhood', 
                               right_on = 'neighbourhood_cleansed')

# tmp_price = gdf_price.groupby('neighbourhood_group')['geometry'].agg(list).reset_index()
# from shapely.ops import unary_union
# tmp_price['geometry'] = tmp_price['geometry'].apply(lambda x: unary_union(x))
# tmp = gdf_price[['neighbourhood', 'neighbourhood_group', 'neighbourhood_group_cleansed', 'price', 'neighbourhood_cleansed', 'price_mean']].merge(tmp_price[['neighbourhood_group', 'geometry']], 
#                       left_on='neighbourhood_group', right_on='neighbourhood_group')

# make map of average prices paid in each neighbourhood of Berlin

style_function = lambda x: {'weight': 0.5,
                           'fillColor': '#8c8c8c',
                           'fillOpacity': 0,
                           'color': 'black'} if pd.isnull(x['properties']['price_mean'])else {'weight': 0.5,
                                                                                             'fillColor': colormap(x['properties']['price_mean']),
                                                                                             'fillOpacity': .90,
                                                                                             'color': 'black'}

highlight_function = lambda x: {'weight': 0.5,
                           'fillColor': '#8c8c8c',
                           'fillOpacity': 0,
                           'color': 'black'} if pd.isnull(x['properties']['price_mean']) else {'weight': 0.9,
                                                                                               'fillColor': colormap(x['properties']['price_mean']),
                                                                                               'fillOpacity': 1,
                                                                                               'color': 'black'}

styles= folium.features.GeoJson(
    gdf_price,
    style_function=style_function,
    highlight_function = highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['neighbourhood_cleansed','price_mean'],
        aliases = ['Neighbourhood: ','Average Price Paid: ']))

### Results/ Visualisations:

In [ ]:
# make folium map centered in Berlin
m = folium.Map(location=[52.51, 13.40], zoom_start=11, tiles='cartodbpositron')

colormap.caption = 'Average Price Paid'
colormap.add_to(m)

m.add_child(styles)

# Save the map
m.save("avg_price_paid.html")

m

In [ ]:
gdf_price

## District Map of Berlin

In [ ]:
# # FOR THE neighbourhood_grouped_cleansed

# reviews_grouped_1 = reviews_neighbourhood[reviews_neighbourhood.year 
#                       >=2019].groupby('neighbourhood_group_cleansed')['price'].agg('mean').reset_index()

# reviews_grouped_1 = reviews_grouped_1.merge(reviews_neighbourhood[['neighbourhood_group_cleansed', 'neighbourhood_cleansed']], 
#                                             left_on='neighbourhood_group_cleansed', right_on='neighbourhood_group_cleansed')

# reviews_grouped_1 = reviews_grouped_1.drop_duplicates('neighbourhood_cleansed').reset_index(drop=True)

# reviews_grouped_1['price_mean'] = reviews_grouped_1.price.apply(lambda x: round(x,2))

# print(reviews_grouped_1.head(), reviews_grouped_1.shape)

# list_group = reviews_grouped_1['neighbourhood_group_cleansed'].unique().tolist()
# reviews_grouped_1['number'] = reviews_grouped_1['neighbourhood_group_cleansed'].apply(lambda x: list_group.index(x))

In [ ]:
gdf_price = geo_df.merge(reviews_grouped, left_on='neighbourhood', 
                               right_on = 'neighbourhood_cleansed')

plt.rcParams['figure.figsize'] = [20, 13]
fig, ax = plt.subplots()

gdf_price.plot(
    ax=ax, 
    alpha=0.2
)

gdf_price.plot(
    ax=ax, 
    column='neighbourhood_group',
    categorical=True, 
    legend=True, 
    legend_kwds={'title': 'Neighbourhood', 'loc': 'upper right'},
    cmap='tab20', 
    edgecolor='black',
    
)

ax.set(
    title='Berlin District/Kiez', 
    aspect=1.3
);

fig.show()


In [ ]:
gdf_price.to_file('dataframe.geojson', driver='GeoJSON')

### Look at prices in larger districts:

In [ ]:
# get only information for those places stayed in 2019 and later
reviews_boxplots = reviews_neighbourhood[reviews_neighbourhood.year >= 
                                        2019]

# get all individual listing data for places with at least 1 review
reviews_boxplots = reviews_boxplots.groupby('listing_id').first()

# plot results as boxplots:
fig = px.box(reviews_boxplots, x = 'neighbourhood_group_cleansed', y = 'price')
fig.update_layout(
    title="Listing Prices in Berlin Districts",
    yaxis_title="Listing Price (per night)",
    xaxis_title = 'District')
fig.update_yaxes(range=[-30,1030])
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Listing_Prices_in_Berlin_Districts.html')

In [ ]:
# Get table of median and mean price information for each district as well:
groups = reviews_boxplots.groupby('neighbourhood_group_cleansed')['price'].agg(['mean',
                                                                                'median']).sort_values(by='mean',
                                                                                                       ascending=False).reset_index()
groups.columns = ['District','Mean Price','Median Price']
groups

## Now, taking location ratings into account:
### Data Preparation:

In [ ]:
reviewed_listings_df = reviews_df[['listing_id',
                                    'comments',
                                    'date',
                                   'year',
                                   'month']].copy().merge(listings_df[['id',
                                                                       'neighbourhood_group_cleansed',
                                                                       'neighbourhood_cleansed',
                                                                       'review_scores_location',
                                                                       'price',
                                                                       'latitude',
                                                                       'longitude']].copy(),
                                                          left_on='listing_id',
                                                          right_on='id',
                                                          how='inner').drop(columns = 'id')

reviews_listings_df = reviewed_listings_df.groupby('listing_id').first()
 
# make dataframes for review and price averages for each district and combine:


location_reviews_df = reviewed_listings_df.groupby('neighbourhood_group_cleansed')[['review_scores_location']].mean()
location_prices_df = reviewed_listings_df.groupby('neighbourhood_group_cleansed')[['price']].mean()
location_reviews = pd.concat([location_reviews_df, location_prices_df], axis=1)
location_reviews.reset_index(inplace=True)
location_reviews.sort_values(by='review_scores_location', ascending=False, inplace = True)
location_reviews.rename(columns = {'review_scores_location': 'Location Rating'}, inplace = True)
location_reviews = location_reviews.reset_index(drop=True)

In [ ]:
# location_reviews = location_reviews.reset_index(drop=True)
# location_reviews.to_csv('location_reviews.csv', index=False)

### Results/Visualisation:

In [ ]:
fig = px.bar(location_reviews, x = 'neighbourhood_group_cleansed', 
             y = 'price', color = 'Location Rating')
fig.update_layout(
    title="Average District Price Relative to Location Rating",
    xaxis_title = "District",
    yaxis_title = "Price per Night ($)"
    )
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Average_District_Price_Relative_to_Location_Rating.html')

## Heatmap of reviews in time (not in post):

In [ ]:
reviews_neighbourhood[['latitude','longitude','date','year','month']]

In [ ]:
# get a dataframe of reviews for use in the heatmap

heatmap_df = reviews_neighbourhood[['latitude','longitude','date','year','month']].copy()
heatmap_df['lat_lon'] = heatmap_df.apply(lambda x: [round(x['latitude'], 7), round(x['longitude'],7)], axis=1)

# prepare lat/lon list of lists for folium
lat_lon_multistep = list(heatmap_df[(heatmap_df.year < 2016)].groupby(['year','month'])['lat_lon'].apply(list))
# get year for each step to label the map with
timestamps = heatmap_df[heatmap_df.year < 2016].groupby(['year',
                                                         'month'])[['date']].first().reset_index()
timestamps = list(timestamps.year)

### Results / visualisation:


In [ ]:
m = folium.Map(location=[52.51, 13.40], zoom_start=11, tiles='cartodbpositron')
heatmap = HeatMapWithTime(data = lat_lon_multistep,
                              index=timestamps, auto_play=True, min_speed = 6)
heatmap.add_to(m)


# Save the map
m.save("growth_of_aribnb.html")
m

# Question 3:
Can we predict the location rating for a listing, and what neighbourhoods have the greatest influence on those ratings?

## Data Preparation:

In [ ]:
listings_df.amenities.unique(), listings_df.host_acceptance_rate.unique(), listings_df.review_scores_location.unique()

In [ ]:
# define the review scores column names:
target_colnames = ['review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value']
df = listings_df.copy()
# clean variables that are strings but should be floats:

df['host_response_rate'] = df.host_response_rate.apply(
    lambda x: x if pd.isnull(x) else float(x.strip('%')))

df['host_acceptance_rate'] = df.host_acceptance_rate.apply(
    lambda x: x if pd.isnull(x) else float(x.strip('%')))

# change date of host_since to num days they have been a host
df['host_days'] = df.host_since.apply(lambda x: (pd.to_datetime('today') - pd.to_datetime(x)).days)
df.drop('host_since', 1, inplace = True)

In [ ]:
# drop all rows that have NA values in the target variable we are using (they all have a similar number of NA values)
df.dropna(subset = ['review_scores_location'], inplace = True)

# make a feature for how many cancellations each listing has had
reviews_df['canceled'] = reviews_df.comments.str.contains('canceled|cancellation')
canceled_df = reviews_df.groupby('listing_id')[['canceled']].sum().astype(int)
df = df.merge(canceled_df, left_on='id', right_on='listing_id')
# More feature engineering:
# print(df.head())

# make features for the number of verifications a host has, the number of amenities, and clean
# the amenities column
df['num_verifications'] = df.host_verifications.apply(
    lambda x: len(x.strip('][').replace("'","").split(', ')))
df['amenities'] = df.amenities.apply(
    lambda x: x.strip('}{').replace("'","").replace("\"", "").split(','))
df['num_amenities'] = df.amenities.apply(
    lambda x: len(x))

# feature for how many missing values a listing has
df['num_null'] = df.isnull().sum(axis=1)
# print(df.head())
# clean all columns with prices
price_cols = ['weekly_price','security_deposit','cleaning_fee', 'extra_people']

df.loc[:,price_cols] = df[price_cols].applymap(
    lambda x: x if pd.isnull(x) else float(x.replace("$", "").replace(",","")))
# get the distance in km that each listing is from the city centre

city_centre_coords = (13.3888599, 52.5170365)
df['dist_to_center'] = df.apply(
    lambda x: geopy.distance.geodesic(city_centre_coords, (x['longitude'],
                                                           x['latitude'])).km, 1)
# print(df.head())

### Geographical Feature Engineering:
Get extra geographical data from OSM, first:

### Distances of properties to closest train station/underground:

In [ ]:
# get the latitude and longitude values for all U-Bahn and S-bahn stations in Berlin
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area[name = "Berlin"];
(node(area)[railway=station];
node(area)[station=subway];
);out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()
# Collect coords into list and plot:
coords =[]
lon_ls = []
lat_ls = []
type_ls = []
name_ls = []
for element in data['elements']:
    if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        station_Sbahn= element['tags']['name'].startswith('S') # startswith() returns True or False
        name = element['tags']['name']
        if lon >=12:
            lon_ls.append(lon)
            lat_ls.append(lat)
            coords.append((lon, lat))
            type_ls.append(station_Sbahn)
            name_ls.append(name)
# Convert coordinates into numpy array
X = np.array(coords)
plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Stations in Berlin')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

# make data frame of the latitude and longitudes:
coord_df = pd.DataFrame({'latitude_value': lat_ls, 
                         'longitude_value': lon_ls, 'sbahn': type_ls})

In [ ]:
loc = 'Station Locations'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

stat_df = gpd.GeoDataFrame(coord_df, geometry=gpd.points_from_xy(coord_df.latitude_value, coord_df.longitude_value))
m = folium.Map(location=[52.51, 13.40], zoom_start=10, tiles='cartodbpositron')
for point in range(0, len(stat_df)):
    folium.CircleMarker([stat_df.loc[point, 'latitude_value'], stat_df.loc[point, 'longitude_value']], radius=3).add_to(m)

style_function = lambda x: {'weight': 0.5,
                           'fillColor': '#8c8c8c',
                           'fillOpacity': 0,
                           'color': 'black'} 

styles= folium.features.GeoJson(
    gdf_price,
    style_function=style_function)

m.add_child(styles)
m.get_root().html.add_child(folium.Element(title_html))

# Save the map
m.save("station_locations.html")

m

In [ ]:
coord_df.to_csv('station.csv', index=False)

In [ ]:
# get all pairwise distances between each listing and stations
import scipy
distances = scipy.spatial.distance.cdist(df[['latitude','longitude']], 
                                         coord_df[['latitude_value','longitude_value']],
                                         metric='seuclidean')

distance_sbahn = scipy.spatial.distance.cdist(df[['latitude','longitude']], 
                                              coord_df[coord_df.sbahn][['latitude_value','longitude_value']],
                                             metric='seuclidean')

distance_ubahn = scipy.spatial.distance.cdist(df[['latitude','longitude']], 
                                              coord_df[coord_df.sbahn == False][['latitude_value',
                                                                                 'longitude_value']],
                                              metric='seuclidean')
# make feature for the distance to closest train station
df['dist_to_station'] = distances.min(axis=1)
df['dist_to_sbahn'] = distance_sbahn.min(axis=1)
df['dist_to_ubahn'] = distance_ubahn.min(axis=1)

In [ ]:
distances[0].min(), distance_sbahn[0].min(), distance_ubahn[0].min()

### Distance to closest Biergarten: (A beer garden (German: Biergarten) is an outdoor area in which beer and food are served, typically at shared tables.)

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area[name = "Berlin"];
(node(area)["amenity"="biergarten"];
);out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()
# Collect coords into list and plot:
lon_ls = []
lat_ls = []
for element in data['elements']:
    if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        if lon >=12:
            lon_ls.append(lon)
            lat_ls.append(lat)
# make data frame of the latitude and longitudes:
coord_df = pd.DataFrame({'latitude_value': lat_ls, 'longitude_value': lon_ls})
# get all pairwise distances between each listing and beer gardens
distances = scipy.spatial.distance.cdist(df[['latitude','longitude']], 
                                         coord_df[['latitude_value','longitude_value']],
                                         metric='seuclidean')
df['dist_to_biergarten'] = distances.min(axis=1)

In [ ]:
# coord_df

In [ ]:
loc = 'Biergarten Locations'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc) 

stat_df = gpd.GeoDataFrame(coord_df, geometry=gpd.points_from_xy(coord_df.latitude_value, coord_df.longitude_value))
m = folium.Map(location=[52.51, 13.40], zoom_start=11, tiles='cartodbpositron')
for point in range(0, len(stat_df)):
    folium.CircleMarker([stat_df.loc[point, 'latitude_value'], stat_df.loc[point, 'longitude_value']], radius=3, color = 'green').add_to(m)

style_function = lambda x: {'weight': 0.5,
                           'fillColor': '#8c8c8c',
                           'fillOpacity': 0,
                           'color': 'black'} 

styles= folium.features.GeoJson(
    gdf_price,
    style_function=style_function)

m.add_child(styles)
m.get_root().html.add_child(folium.Element(title_html))

# Save the map
m.save("biergarten.html")

m

In [ ]:
coord_df.to_csv('biergarten.csv', index=False)

In [ ]:
# list(listings_df.columns[listings_df.isnull().sum()/listings_df.shape[0] >= .95])

### Numeric variables:

In [ ]:
# make a dataframe with only the numeric variables in df
num_vars = df.select_dtypes(exclude=['object'])

# get a list of cols to drop, including those that are missing more than 95% of the values, as well as the ID variables
# and lat, lon which don't have relevance to this prediction

cols_to_drop = list(num_vars.columns[num_vars.isnull().sum()/num_vars.shape[0] >= .95]) + ['id',
                                                                                           'scrape_id',
                                                                                           'host_id',
                                                                                           'latitude',
                                                                                           'longitude',
                                                                                           'host_listings_count',
                                                                                           'host_total_listings_count']
# drop the cols_to_drop columns:
num_vars = num_vars.drop(columns = cols_to_drop)
# make histograms
# change the iloc argument [:,:9] to whichever variables you want to look at
# (not including all histograms for lack of space)

num_vars.iloc[:,:9].hist(figsize=(15,8))
plt.show()

In [ ]:
# make features for certain columns which we don't want to impute, but still want to know which values are null
# in case they are significant

num_vars['weekly_price_null'] = num_vars.weekly_price.isnull()
num_vars.drop('weekly_price', 1, inplace = True)

num_vars['deposit_null'] = num_vars.security_deposit.isnull()
num_vars['cleaning_fee_null'] = num_vars.cleaning_fee.isnull()

num_vars.drop('cleaning_fee',1, inplace = True)

# make variables for acceptance and response rate null values, as they are probably important, and we don't
# want to throw out that information

num_vars['host_acceptance_rate_null'] = num_vars.host_acceptance_rate.apply(
    lambda x: pd.isnull(x))
num_vars['host_response_rate_null'] = num_vars.host_response_rate.apply(
    lambda x: pd.isnull(x))
# IMPUTATION: Not using currently as we are only interested in the location columns. However,
# if we would like to predict other ratings, can use these imputation methods

# impute the following columns to median value
impute_median_cols = ['bedrooms','beds','bathrooms',
                      'reviews_per_month','security_deposit','host_days',
                     'price']
num_vars.loc[:,impute_median_cols] = num_vars[impute_median_cols].fillna(num_vars[impute_median_cols].median())

# # now impute the columns to the mean or median value
num_vars['host_acceptance_rate'] = num_vars.host_acceptance_rate.fillna(num_vars.host_acceptance_rate.mean())
num_vars['host_response_rate'] = num_vars.host_response_rate.fillna(num_vars.host_response_rate.mean())
num_vars['host_days'] = num_vars.host_days.fillna(num_vars.host_days.median())
# from stackoverflow: https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=30):
    '''Get the top correlations wrt absolute value'''
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]
# look at the top correlations in the dataset so we can remove redundant variables
corr_df = num_vars.corr()
get_top_abs_correlations(corr_df).head(10)

In [ ]:
# remove the following columns as they are too highly correlated with other columns
redundant_cols = ['bathrooms', 'accommodates',
                  'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
                 'minimum_minimum_nights','maximum_minimum_nights',
                  'maximum_maximum_nights',
                 'minimum_maximum_nights', 'availability_60', 'availability_90',
                  'number_of_reviews_ltm']

num_vars.drop(columns = redundant_cols, inplace = True)

In [ ]:
num_vars.columns, num_vars.shape

In [ ]:
# from sklearn import preprocessing
# min_max_scaler = preprocessing.MinMaxScaler()
# cols = ['host_response_rate', 'host_acceptance_rate', 'bedrooms', 'beds',
#        'price', 'security_deposit', 'guests_included', 'extra_people',
#        'minimum_nights', 'maximum_nights', 'availability_30',
#        'availability_365', 'number_of_reviews', 'review_scores_rating',
#        'review_scores_accuracy', 'review_scores_cleanliness',
#        'review_scores_checkin', 'review_scores_communication', 'review_scores_value',
#        'calculated_host_listings_count',
#        'calculated_host_listings_count_entire_homes',
#        'calculated_host_listings_count_private_rooms',
#        'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
#        'host_days', 'canceled', 'num_verifications', 'num_amenities',
#        'num_null', 'dist_to_center', 'dist_to_station', 'dist_to_sbahn',
#        'dist_to_ubahn', 'dist_to_biergarten', 'weekly_price_null',
#        'deposit_null', 'cleaning_fee_null', 'host_acceptance_rate_null',
#        'host_response_rate_null']
# num_vars[cols] = min_max_scaler.fit_transform(num_vars[cols])


In [ ]:
num_vars.review_scores_location

### Categorical Variables:

In [ ]:
# get a dataframe of just the categorical variables
cat_vars = df.select_dtypes(include=['object']).copy()

# take only columns of interest
cat_vars = cat_vars[['host_response_time',#'host_neighbourhood','host_location',
         'host_is_superhost','host_has_profile_pic','host_identity_verified',
          'neighbourhood_cleansed','neighbourhood_group_cleansed',
                    'property_type','room_type','bed_type','requires_license',
               'instant_bookable','cancellation_policy','amenities']]
# Clean up the amenities column, and make dummy variables for the important amenities

amenity_list_list = cat_vars.amenities.to_list()
amenity_list = [item for sublist in amenity_list_list for item in sublist]
# get the counts of each amenity by how many times it appears
counts = pd.Series(amenity_list).value_counts()
values = cat_vars.amenities.copy()

# get a series of the top 20 advertised amenities
top_amenities = pd.Series(counts.iloc[:20].index)
# set all listings that have any of those top 20 amenities to true (to make the mask)
mask = pd.Series([(top_amenities.any() in value) for value in values])
# mask out amenities that are not in the top 20 most popular
values[~mask] = 'other'
# add a dummy column for each of the top 20 amenities
for col in top_amenities:
    values = [col in amenity_ls for amenity_ls in cat_vars.amenities] # returns bool
    cat_vars[col] = values
    
# drop amenities column
cat_vars.drop(columns = 'amenities', inplace = True)
# print(cat_vars.head())
# make dummy variables for neighbourhoods (only those with at least 200 counts, otherwise there are too many)

values = cat_vars.neighbourhood_cleansed.copy()
counts = cat_vars.neighbourhood_cleansed.value_counts()
mask = values.isin(counts[counts > 200].index)
# print(mask[:40])
values[~mask] = "other"
# make dummy columns from the top neighbourhoods:
dummies = pd.get_dummies(values)
dummies.columns = pd.Series(dummies.columns).apply(lambda x: 'neighbourhood_cleansed_' + x)
dummies.drop(columns = 'neighbourhood_cleansed_other', inplace = True)

# add dummy variables back into cat_vars df
cat_vars = pd.concat([cat_vars.drop('neighbourhood_cleansed', axis=1), dummies], 1)
# get dummies for other important variables:

cat_vars = pd.get_dummies(cat_vars, 
                          columns = ['host_response_time','neighbourhood_group_cleansed',
                                    'room_type','bed_type', 'cancellation_policy'],
                          drop_first = True)
# change variables that just have a t or f to bools
tf_vars = ['requires_license','instant_bookable',
         'host_is_superhost','host_has_profile_pic','host_identity_verified']
cat_vars.loc[:,tf_vars] = cat_vars[tf_vars].applymap(lambda x: True if x=='t' else False)
# make dummy variables for the most common property types

values = cat_vars.property_type.copy()
counts = cat_vars.property_type.value_counts()
mask = values.isin(counts[counts > 3].index)
values[~mask] = "other"

dummies = pd.get_dummies(values)
dummies.columns = pd.Series(dummies.columns).apply(lambda x: 'property_type_' + x)
dummies.drop(columns = 'property_type_other', inplace = True)

cat_vars = pd.concat([cat_vars.drop('property_type', axis=1), dummies], 1)

In [ ]:
cat_vars.columns, cat_vars.shape

In [ ]:
num_vars.review_scores_value

# Modelling:
### Fit a logistic regression model:

In [ ]:
def coef_weights(coefficients, X_train):
    '''
    Inputs:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    Output:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = coefficients
    coefs_df['abs_coefs'] = np.abs(coefficients)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df
# reset indices so we can combine numeric and categorical variables
cat_vars.reset_index(drop = True, inplace = True)
num_vars.reset_index(drop = True, inplace = True)

# combine dataframes to get full dataframe
fill_df = pd.concat([cat_vars, num_vars], axis = 1)

# change the boolean values to integers
bool_df = fill_df.select_dtypes(include='bool').astype(int)
fill_df[bool_df.columns] = bool_df

### Focus only on features relevant for predicting LOCATION ratings:
(Can change this if you want to use other columns)

In [ ]:
# location_cols = list(fill_df.columns[pd.Series(fill_df.columns).str.contains('location|neighbourhood|dist|room_type|property_type')])

# # add any other features we want to include in model:
# location_cols = location_cols + ['host_is_superhost','host_response_rate_null']
# fill_df = fill_df[location_cols]
# look at highly correlated columns so we can reduce multi-collinearity, problematic in linear models

corr_df = fill_df.corr()
get_top_abs_correlations(corr_df, n = 11)

In [ ]:
# drop the columns that are too highly correlated with other columns in dataframe

redundant_cols = ['neighbourhood_cleansed_Neu Lichtenberg', 
                  'neighbourhood_cleansed_Schöneberg-Nord',
                  'room_type_Hotel room','room_type_Shared room', 'dist_to_ubahn',
                  'neighbourhood_cleansed_Schillerpromenade',
                 'neighbourhood_cleansed_Schöneberg-Süd',
                  'neighbourhood_cleansed_Reuterstraße',
                  'neighbourhood_cleansed_Alexanderplatz', 'neighbourhood_cleansed_Rixdorf',
                 'neighbourhood_cleansed_Neuköllner Mitte/Zentrum',
                  'neighbourhood_cleansed_Frankfurter Allee Süd FK',
                 'neighbourhood_cleansed_Tempelhof',
                  'neighbourhood_cleansed_Tempelhofer Vorstadt',
                 'neighbourhood_cleansed_Brunnenstr. Nord']



fill_df = fill_df.drop(columns = redundant_cols)
# examine distribution of target column:




In [ ]:
fig = plt.figure(figsize = (8,6))
ax = fig.gca()

fill_df['review_scores_value'].hist(ax = ax)
plt.title('Histogram of Location Ratings')
plt.ylabel('Count')
plt.xlabel('Rating')
plt.show()

In [ ]:
fig = px.histogram(fill_df, x = 'review_scores_value')
fig.update_layout(
    title="Histogram of Listing Rating",
    xaxis_title = "Rating",
    yaxis_title = "Count"
    )
fig.show()

# Saving the plot
plotly.offline.plot(fig, filename='Histogram_of_Location_Rating.html')

In [ ]:
fill_df.columns

In [ ]:
from sklearn import preprocessing

# set X dataframe
X = fill_df.drop(columns = ['review_scores_value'], axis = 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# X[['dist_to_center', 'dist_to_station', 'dist_to_sbahn', 'dist_to_biergarten']] = min_max_scaler.fit_transform(X[['dist_to_center', 'dist_to_station', 'dist_to_sbahn', 'dist_to_biergarten']])

# set target column, predicting either rating == 10 or not
y = fill_df['review_scores_value']
y = (y < 10).astype(int)
# see that 70% of ratings = 10 and the other 30% are less than 10
y.value_counts(normalize=True)

In [ ]:
X =X.fillna(X.mean())

In [ ]:
# check that there are no NA values
print('There are', X.isnull().sum().sum(), 'null values in the dataset')

In [ ]:
y.value_counts()

In [ ]:
get_top_abs_correlations(X, n = 11)
X = X.drop(['Cooking basics', 'Oven'], axis=1)

In [ ]:
# # # split into train and test sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)
# # # fit a logistic regression model, classifying whether rating is 10 or not
# # # lr_model = sm.Logit(y_train, X_train).fit(inflation='probit')
# import warnings

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression

# # pipe = Pipeline([('classifier' , RandomForestClassifier())])
# # # pipe = Pipeline([('classifier', RandomForestClassifier())])

# # # Create param grid.

# # param_grid = [
# #     {'classifier' : [LogisticRegression()],
# #      'classifier__penalty' : ['l1', 'l2'],
# #     'classifier__C' : np.logspace(-4, 4, 20),
# #     'classifier__solver' : ['liblinear']},
# #     {'classifier' : [RandomForestClassifier()],
# #     'classifier__n_estimators' : list(range(10,101,10)),
# #     'classifier__max_features' : list(range(6,32,5))}
# # ]

# # # Create grid search object

# # lr_model = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True)



# param_grid = {    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
#     'penalty': ['l1', 'l2']
#              }
# lr_model = GridSearchCV(LogisticRegression(random_state=0,solver='liblinear'), param_grid = param_grid, cv = 5, verbose=True)

# lr_model.fit(X_train, y_train)

In [ ]:
t = False

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
skf = StratifiedKFold(n_splits=5)


if t==False:
    X_tmp = X
    y_tmp = y
    t = True
else:
    X= X_tmp
    y = y_tmp

fold_no=1
lr_model = LogisticRegression(random_state=200,solver='newton-cg', penalty='l2', class_weight='balanced', 
                              max_iter=700, warm_start=True, multi_class='ovr', C=1)
X, X_test, y, y_test = train_test_split(X, y, test_size = .30, random_state=42)
for train_index,test_index in skf.split(X, y):
    if fold_no == 5: 
        continue
    train = X.iloc[train_index,:]
    test = X.iloc[test_index,:]
    
    lr_model.fit(train, y.iloc[train_index])
    
    y_pred_train_prob = lr_model.predict(train)
    y_pred_test_prob = lr_model.predict(test)
    # choose threshold of .5 for probability threshold to classify into 10 rating
    y_pred_test = (y_pred_test_prob >= .5).astype(int)
    y_pred_train = (y_pred_train_prob >= .5).astype(int)


    print('Train Accuracy:', accuracy_score(y.iloc[train_index], y_pred_train))
    print('Test Accuracy:', accuracy_score(y.iloc[test_index], y_pred_test))
    
    lr_precision, lr_recall, _ = precision_recall_curve(y.iloc[test_index], y_pred_test_prob)
    lr_f1, lr_auc = f1_score(y.iloc[test_index], y_pred_test), auc(lr_recall, lr_precision)

    # summarize scores
    # note: a baseline AUC for precision-recall would be about .3 since 30% of the instances are class 1
    print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
    fold_no += 1

In [ ]:
# get test and train predictions:
y_pred_test_prob = lr_model.predict(X_test)
y_pred_train_prob = lr_model.predict(X_train)
# choose threshold of .5 for probability threshold to classify into 10 rating
y_pred_test = (y_pred_test_prob >= .5).astype(int)
y_pred_train = (y_pred_train_prob >= .5).astype(int)


print('Train Accuracy:', accuracy_score(y_train, y_pred_train))
print('Test Accuracy:', accuracy_score(y_test, y_pred_test))

In [ ]:
# Accuracy doesn't tell us much, since the classes are quite imbalanced, so let's look at precision
# recall curves and f1 score
lr_precision, lr_recall, _ = precision_recall_curve(y_test, y_pred_test_prob)
lr_f1, lr_auc = f1_score(y_test, y_pred_test), auc(lr_recall, lr_precision)

# summarize scores
# note: a baseline AUC for precision-recall would be about .3 since 30% of the instances are class 1
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))

# Logistic: f1=0.726 auc=0.786

In [ ]:
from sklearn.metrics import confusion_matrix

#Generate the confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred_test)

print(cf_matrix)

## Graphs for results

In [ ]:
import seaborn as sns
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g', annot_kws={"fontsize":22})
ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['0','1'])
ax.yaxis.set_ticklabels(['0','1'])
## Display the visualization of the Confusion Matrix.


plt.savefig('confusion_matrix.png', bbox_inches='tight')
plt.show()

In [ ]:
feat_importances = pd.Series(lr_model.coef_[0], index=X.columns)
df = pd.DataFrame(feat_importances).reset_index(drop=False)
df.columns = ['feat', 'imp']
df['abs_coef'] = df['imp'].apply(lambda x: abs(x))
df.sort_values(by='abs_coef', ascending=True, inplace = True)
df = df.reset_index(drop=True)
df['geo'] = pd.Series(df['feat'].values).str.contains('location|neighbourhood|dist|room_type|property_type')
df = df[df['geo'] == True]
# df = df.drop(['host_response_time_within a day', 'host_response_time_within a few hour', 'host_response_time_within an hour'], axis=1)
df=df.head(15)
fig = px.bar(x=df.imp, y=df.feat, orientation='h')
fig.update_layout(
    yaxis_title="Feature",
    xaxis_title = "Effect Size (Normalized)",
    title = "Top Effect Sizes for Predicting Location Rating")

plotly.offline.plot(fig, filename='feat_imp.html')
fig.show()

In [ ]:
 from sklearn.metrics import PrecisionRecallDisplay

disp = PrecisionRecallDisplay(precision=lr_precision, recall=lr_recall)
disp.plot()
plt.show()
